In [0]:
%sql
CREATE OR REPLACE TABLE sandbox.t_weetv.po_activation_mart
USING DELTA
PARTITIONED BY (date_ym)
AS
WITH T101 AS (
    /* KMS 관련 테이블 */
    SELECT 
      date_ym,
      mac,
      try_cast(po_date AS DATE) AS po_date,
      po_country_code,
      po_type,
      po_board_maker,
      po_odm,
      po_brand,
      po_country,
      po_year,
      po_platform
    FROM kic_data_ods.kms.kms_wee_tv
),
T102 AS (
    /* ACTIVATION: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.activ_country_code,
        s.activ_date,
        s.activ_platform_code,
        s.activ_product_code,
        s.activ_sales_model
    FROM (
        SELECT
            T102.mac_addr,
            T102.Cntry_CODE   AS activ_country_code,
            DATE(T102.crt_date) AS activ_date,
            T102.Platform_code AS activ_platform_code,
            T102.Product_CODE  AS activ_product_code,
            T102.Sales_Model   AS activ_sales_model,
            ROW_NUMBER() OVER (
                PARTITION BY T102.mac_addr
                ORDER BY T102.crt_date ASC, T102.last_chg_date ASC, T102.he_etl_dt ASC
            ) AS rn
        FROM aic_data_ods.tlamp.activation_date T102
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T102.mac_addr
        )
    ) s
    WHERE s.rn = 1
),
T103 AS (
    /* 앱 브랜드: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.brand_name
    FROM (
        SELECT
            T103.mac_addr,
            T103.brand_name,
            ROW_NUMBER() OVER (
                PARTITION BY T103.mac_addr
                ORDER BY T103.src_file_date ASC, T103.he_etl_dt ASC
            ) AS rn
        FROM aic_data_ods.tlamp.apps_device_brand T103
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T103.mac_addr
        )
    ) s
    WHERE s.rn = 1
)
SELECT
    -- 파티션 키
    T101.date_ym,

    -- PO 측 주요 컬럼
    T101.mac,
    T101.po_date,
    T101.po_country_code,
    T101.po_type,
    T101.po_board_maker,
    T101.po_odm,
    T101.po_brand,
    T101.po_country,
    T101.po_year,
    T101.po_platform,

    -- ACTIVATION 측
    T102.activ_date,
    T102.activ_country_code,
    T102.activ_platform_code,
    T102.activ_product_code,
    T102.activ_sales_model,

    -- BRAND 파싱
    SPLIT_PART(T103.brand_name, '_', 1)   AS activ_board_maker,
    SPLIT_PART(T103.brand_name, '_', 2)   AS activ_odm,
    SPLIT_PART(T103.brand_name, '_', 3)   AS activ_brand,

    -- 비교 지표
    CASE
        WHEN T101.po_country_code IS NULL THEN 'po_country_code_is_null'
        WHEN T102.activ_country_code IS NULL THEN 'not_yet_activated'
        WHEN T101.po_country_code <> T102.activ_country_code THEN 'N'
        ELSE 'Y'
    END                                      AS is_same_country,
    DATE_DIFF(T102.activ_date, T101.po_date) AS day_diff,
    
    current_timestamp() as he_etl_dt
FROM T101
LEFT JOIN T102
  ON T102.mac_addr = T101.mac
LEFT JOIN T103
  ON T103.mac_addr = T101.mac
;

In [0]:
%sql
CREATE OR REPLACE VIEW sandbox.t_weetv.v_po_activation_detail AS
SELECT
    a.mac                                            AS MAC,
    a.po_type                                        AS PO_Type,

    -- PO_Platform 매핑
    CASE a.po_platform
      WHEN '00' THEN 'C20P'
      WHEN '01' THEN 'C20L'
      WHEN '10' THEN 'C22P'
      WHEN '11' THEN 'C22L'
      WHEN '12' THEN 'C22H'
      WHEN '13' THEN 'C22T'
      WHEN '21' THEN 'C23L'
      WHEN '30' THEN 'C23T'
      WHEN '31' THEN 'C23U'
      WHEN '33' THEN 'C23T'
      WHEN '41' THEN 'C23V'
      WHEN '51' THEN 'C23U'
      WHEN '60' THEN 'C25P'
      WHEN '65' THEN 'C25H'
      WHEN '66' THEN 'C25H'
      WHEN '70' THEN 'C24T'
      ELSE a.po_platform
    END                                             AS PO_Platform,

    -- 국가코드 → 국명 (없으면 원래 코드 유지)
    COALESCE(b.country, a.po_country_code)          AS PO_Country,

    -- PO_Board_Maker 매핑
    CASE a.po_board_maker
      WHEN 'TT' THEN 'TOPTECH'
      WHEN 'MS' THEN 'MORESTAR'
      WHEN 'CV' THEN 'CULTRAVIEW'
      WHEN 'SP' THEN 'SILICONPLAYER'
      WHEN 'HK' THEN 'HIKEEN'
      ELSE a.po_board_maker
    END                                             AS PO_Board_Maker,

    a.po_odm                                        AS PO_ODM,
    a.po_brand                                      AS PO_Brand,
    a.po_date                                       AS PO_Date,

    -- Activation 측
    a.activ_platform_code                           AS Act_Platform,
    COALESCE(c.country, a.activ_country_code)       AS Act_Country,

    -- Act_Board_Maker 매핑
    CASE a.activ_board_maker
      WHEN 'TT' THEN 'TOPTECH'
      WHEN 'MS' THEN 'MORESTAR'
      WHEN 'CV' THEN 'CULTRAVIEW'
      WHEN 'SP' THEN 'SILICONPLAYER'
      WHEN 'HK' THEN 'HIKEEN'
      ELSE a.activ_board_maker
    END                                             AS Act_Board_Maker,

    a.activ_odm                                     AS Act_ODM,
    a.activ_brand                                   AS Act_Brand,
    a.activ_date                                    AS Act_Date,
    a.activ_sales_model                             AS Act_salesModel,

    a.date_ym                                       AS PO_date_ym,
    a.he_etl_dt                                     AS he_etl_dt
FROM sandbox.t_weetv.po_activation_mart a
LEFT JOIN sandbox.t_weetv.country_code_name_map b
       ON a.po_country_code = b.country_cd
LEFT JOIN sandbox.t_weetv.country_code_name_map c
       ON a.activ_country_code = c.country_cd
;

In [0]:
%sql
select 
  max(PO_Date), max(Act_Date), max(he_etl_dt)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  -- and Act_Date is null

In [0]:
%sql
select 
  count(1), count(distinct MAC)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  -- and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM)
-- group by
--   PO_date_ym

In [0]:
%sql
create or replace view sandbox.z_yeswook_kim.rbt_mac
AS
(
select w60.*, 'webos60' as webos_version
from aic_data_private.tlamp.rbt_mac_addr_webos60 w60
union all
select w22.*, 'webos22' as webos_version
from aic_data_private.tlamp.rbt_mac_addr_webos22 w22
union all
select w23.*, 'webos23' as webos_version
from aic_data_private.tlamp.rbt_mac_addr_webos23 w23
union all
select w24.*, 'webos24' as webos_version
from aic_data_private.tlamp.rbt_mac_addr_webos24 w24
union all
select w25.*, 'webos25' as webos_version
from aic_data_private.tlamp.rbt_mac_addr_webos25 w25
)

In [0]:
%sql
with tb0 as (
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  -- and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM)
), tb1 as (
  select 
    mac_addr as MAC_ORIGIN
    , mac_addr_hashed as MAC
  from sandbox.z_yeswook_kim.rbt_mac
  union all 
  select 
     mac_addr as MAC_ORIGIN
     , aic_data_ods.tlamp.hash_mac(mac_addr) as MAC
  from aic_data_private.tlamp.activation_date
), tb2 as (
  select 
    distinct MAC_ORIGIN, MAC
  from tb1
)
select *
from (
  select 
    tb2.mac_origin
    , tb0.*
    , DATEDIFF(tb0.Act_Date, tb0.PO_Date) AS day_diff
    , row_number() over (order by PO_date_ym, PO_Date, tb2.mac_origin) as rn
  from tb0 
  left join tb2 USING (MAC)
) t
where rn between 0 and 800000
order by PO_date_ym, PO_Date, mac_origin